In [2]:
import pickle
import pandas as pd
import re
import numpy as np

In [3]:
# 读取
hangzhou_ershou1 = pickle.load(open( './hangzhou_ershou1.pkl', 'rb'))
hangzhou_ershou2 = pickle.load(open( './hangzhou_ershou2.pkl', 'rb'))
# hangzhou_new = pickle.load(open( './hangzhou_new.pkl', 'rb'))

In [7]:
hangzhou_ershou = hangzhou_ershou1.copy()
hangzhou_ershou.update(hangzhou_ershou2)

In [11]:
l = []
for key in hangzhou_ershou.keys():
    l.append(len(hangzhou_ershou[key].keys()))
    
print('爬取房屋的总数：',sum(l))

爬取房屋的总数： 35942


In [13]:
hangzhou_ershou_df = pd.DataFrame(columns = pd.DataFrame(hangzhou_ershou['locationcuiyuanpg1']).index)
for i in hangzhou_ershou.keys():
    temp = pd.DataFrame(hangzhou_ershou[i]).T
    hangzhou_ershou_df = pd.concat([hangzhou_ershou_df, temp], ignore_index = True)

In [15]:
# 发现dataframe中每个元素都是一个列表（有可能为空）
def extract_0(x):
    """
    提取列表中第一个元素，若列表为空，则返回none
    """
    try:
        return x[0]
    except:
        return None   

In [16]:
df_hangzhou = hangzhou_ershou_df.copy()
# dataframe中每一列数据都从列表中提取出来
for col in df_hangzhou.columns:
    df_hangzhou[col] = df_hangzhou[col].apply(extract_0)

In [17]:
df_hangzhou.head()

,house_name,total_price,total_price_desc,unit_price,info,location1
0,翠苑四区,220,万,"36,424元/平",2室1厅 | 60.4平米 | 南 | 简装 | 7层 | 板楼,翠苑
1,翠苑四区,225,万,"42,381元/平",2室1厅 | 53.09平米 | 南 | 简装 | 7层 | 板楼,翠苑
2,翠苑二区,248,万,"41,217元/平",3室1厅 | 60.17平米 | 东南 | 简装 | 7层 | 板楼,翠苑
3,翠苑逸景湾,360,万,"41,252元/平",3室1厅 | 87.27平米 | 南 | 简装 | 26层 | 板塔结合,翠苑
4,枫华府第,880,万,"67,646元/平",3室2厅 | 130.09平米 | 南 | 精装 | 10层 | 板塔结合,翠苑


对总价进行处理

In [19]:
df_hangzhou.total_price = df_hangzhou.total_price.astype('float')

In [21]:
# total_price 只有一个单位：万
# df_hangzhou.total_price_desc.unique()

对单价进行处理

In [22]:
df_hangzhou.unit_price = df_hangzhou.unit_price.str.extract(r'([\d,]+)').replace(',','', regex = True).astype('float')[0]

对面积进行处理

In [25]:
df_hangzhou['area'] = df_hangzhou.loc[:,'info'].str.extract('([\d.]+)平米')

数据存储

In [26]:
df_hangzhou.to_csv('hz_ershou_clean.csv')

In [28]:
df_hangzhou.unit_price.describe()

count     35942.000000
mean      39625.255050
std       20116.935677
min        4016.000000
25%       25115.250000
50%       35827.000000
75%       48535.750000
max      130000.000000
Name: unit_price, dtype: float64

In [29]:
df_hangzhou[df_hangzhou.unit_price == 4016.000000]

,house_name,total_price,total_price_desc,unit_price,info,location1,area
35303,龙锦苑,35.8,万,4016.0,2室2厅 | 89.16平米 | 南 | 毛坯 | 5层 | 板塔结合,临安,89.16
